In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load your dataset
df = pd.read_csv('generated_dataset.csv')

In [ ]:
# Encode categorical variables
df['UserGender'] = df['UserGender'].map({'M': 0, 'F': 1})

In [ ]:
# Tokenize and pad liked topics
max_topics_length = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['LikedTopics'])
topics_seq = tokenizer.texts_to_sequences(df['LikedTopics'])
topics_pad = pad_sequences(topics_seq, maxlen=max_topics_length)

In [ ]:
# Split the data
X = [df['UserAge'], df['UserGender'], topics_pad]
y = np.ones(len(df))  # Binary label: 1 for 'liked', assuming all entries are liked
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)